#1.문헌 분류와 감성 분석
감성이 타켓 범주이다.

In [ ]:
!pip install konlpy

In [ ]:
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt -O ratings_train.txt
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt -O ratings_test.txt

--2024-04-10 12:50:19--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt [following]
--2024-04-10 12:50:19--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    



In [ ]:
import pandas as pd
pd.read_csv('ratings_test.txt', sep='\t').head(5)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')#탭으로 값이 구분된 파일
print(train_df.shape)
train_df = train_df.iloc[0:1500, :]

train, test = train_test_split( train_df, test_size=0.2)
train_df.head(7)

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0


In [ ]:
from konlpy.tag import Okt
twitter = Okt() #open korean text 형태소분석기
def tw_tokenizer(text):
    tokens_ko = twitter.nouns(text) #명사만 추출
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vect = CountVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게" ],
                             min_df=10 )
tfidf_vect.fit(train['document'])  #train의 명사를 뽑은 것까지
tfidf_matrix_train = tfidf_vect.transform(train['document'])
tfidf_matrix_test = tfidf_vect.transform(test['document'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
tfidf_vect.vocabulary_.keys()

dict_keys(['감독', '영화', '돈', '진짜', '감동', '임', '스토리', '명작', '왜', '드라마', '제발', '요즘', '웃음', '정말', '연기', '사람', '시리즈', '평점', '끝', '남자', '감정', '개', '쓰레기', '보고', '점', '더', '중', '조금', '그냥', '거', '역시', '뿐', '안', '건', '내용', '자체', '완전', '한번', '뭐', '시간', '뭔가', '주인공', '기억', '느낌', '배우', '만', '마지막', '때', '장면', '줄', '내내', '재미', '말', '최악', '듯', '하나', '무슨', '좀', '생각', '제일', '정도', '움', '작품', '이야기', '사랑', '것', '수', '내', '최고', '액션', '연출', '음악', '도', '이건', '볼', '별로', '또', '다시', '소재', '이해', '처음', '걸', '함', '일본', '결말', '실망', '여자', '기', '전', '꼭', '난', '전개', '현실', '기분', '인간', '인생', '지금', '속', '못', '때문', '보기'])

In [ ]:
#RF 이용하여 감성 분석 Classification 수행.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#n_estimators: RF에 들어가는 tree의 개수
rf = RandomForestClassifier(random_state=0, n_estimators=400)
rf.fit(tfidf_matrix_train , train['label']  )  #모델링
preds1 = rf.predict(tfidf_matrix_test) #테스트셋으로 감성예측
print('Accuracy: ',accuracy_score(test['label'],preds1))#정답율

Accuracy:  0.6233333333333333


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test['label'],preds1))

              precision    recall  f1-score   support

           0       0.60      0.70      0.65       147
           1       0.66      0.55      0.60       153

    accuracy                           0.62       300
   macro avg       0.63      0.62      0.62       300
weighted avg       0.63      0.62      0.62       300



#2. 감성 스코어

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')#탭으로 값이 구분된 파일
print(train_df.shape)
train_df = train_df.iloc[10000:13000, :]

train, test = train_test_split( train_df, test_size=0.2)
train_df.head(7)

(150000, 3)


,id,document,label
10000,9246662,최고의 2d영화가 아닌가싶으다....!,1
10001,3207978,카리스마작렬~역대퍼니셔시리즈1.2.3中가장좋은액션수작!!,1
10002,8117078,음..별로다..정말로....,0
10003,9782055,"하....정말 말이 안나오는 영화. 내용, 연출, 카메라, 연기까지 뭐 하나 제대로...",0
10004,8384815,"액션보다는 드라마....누미라파스, 콜린패럴의 감정연기가 정말 굳인 영화",1
10005,7252400,잔잔한 감동을 불러 일으키는 영화... 그래 아직 늦지 않았다~!!,1
10006,10059624,아 정말 이런 힐링영화를 상영하는 영화관이 한 두곳밖에 없다는게 정말 안타깝고 대한...,1


In [ ]:
train_df.label.value_counts()

label
0    1526
1    1474
Name: count, dtype: int64

In [ ]:
train_pos = train[train.label==1]
train_pos.label.value_counts()

label
1    1181
Name: count, dtype: int64

In [ ]:
from konlpy.tag import Okt
twitter = Okt() #open korean text 형태소분석기
def tw_tokenizer(text):
    tokens_ko = twitter.nouns(text) #명사만 추출
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게","건",'거' ],
                             min_df=5 )
tfidf_vect.fit(train_pos['document'])  #train의 명사를 뽑은 것까지
tfidf_matrix_train = tfidf_vect.transform(train_pos['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
list( tfidf_vect.vocabulary_.keys() )[1:10]

['우리', '영화', '그것', '정말', '두', '삶', '대한', '감동', '점']

In [ ]:
import pandas as pd

dtm_tfidf_df = pd.DataFrame(tfidf_matrix_train.toarray(), columns=tfidf_vect.get_feature_names_out(), index = train_pos.index)
print(dtm_tfidf_df.sum().sort_values(ascending=False))
words = dtm_tfidf_df.max().sort_values(ascending=False)[100:130]  #[ 30:100 ], [100:130], [150:]
words.index.tolist()

영화    132.359564
정말     43.582858
최고     39.813303
감동     36.778452
진짜     32.740503
         ...    
나중      2.355130
집중      2.303258
슬픔      2.293072
자신      2.253485
배경      2.240091
Length: 225, dtype: float64


['눈물',
 '가족',
 '대한',
 '정도',
 '공포',
 '주제',
 '한국영',
 '우리나라',
 '철학',
 '표현',
 '준',
 '위',
 '걸작',
 '추억',
 '만',
 '나름',
 '몰입',
 '때문',
 '아이',
 '그냥',
 '한번',
 '매력',
 '음악',
 '인생',
 '왜',
 '삶',
 '날',
 '애니',
 '음',
 '저']

In [ ]:
tf_vect = CountVectorizer(tokenizer=tw_tokenizer,
                             stop_words=["그", "나", "이", "게","건",'거' ],
                             vocabulary = words.index.tolist(),
                             min_df=5 )
tf_vect.fit(train['document'])  #train의 명사를 뽑은 것까지
tf_matrix_train = tf_vect.transform(train['document'])
tf_matrix_test = tf_vect.transform(test['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
#RF 이용하여 감성 분석 Classification 수행.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#n_estimators: RF에 들어가는 tree의 개수
rf = RandomForestClassifier(random_state=0, n_estimators=300)
rf.fit(tf_matrix_train , train['label']  )  #모델링
preds1 = rf.predict(tf_matrix_test) #테스트셋으로 감성예측
print('Accuracy: ',accuracy_score(test['label'],preds1))#정답율

Accuracy:  0.535


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test['label'],preds1))

              precision    recall  f1-score   support

           0       0.53      0.93      0.67       307
           1       0.62      0.12      0.21       293

    accuracy                           0.54       600
   macro avg       0.57      0.53      0.44       600
weighted avg       0.57      0.54      0.44       600



In [ ]:
#긍정 Score
dtm_tf_df = pd.DataFrame(tf_matrix_train.todense(), columns =words.index.tolist(), index=train.index )

pos_score = pd.DataFrame( dtm_tf_df.sum(axis=1).sort_values(ascending=False), columns=["score"] )
pos_score["id"] = pos_score.index

train["id"] = train.index
sentiment = pd.merge(pos_score, train, on="id")

In [ ]:
sentiment

,score,id,document,label
0,5,11664,"1,2화는 지금봐도 너무재미잇습니다, 어렷을때에는 정규방송에서 크리스마스날 꼭해줫는...",1
1,4,10858,"솔직히 제목때문에 호기심 땡겨서 대충 대충 봤는데, 기대이상이다. 평점 엉망으로 준...",1
2,3,12298,어떻게 이런 막장 공포 시나리오를 썼을까? 피터잭슨이 간간히 이런 B급 영화도 만들...,1
3,3,12180,와우.. 십점 준게 언제였지.. 마지막에 너무 감동적이다.. 많은것을 다시금 깨닫게...,1
4,3,10685,"주인공도 매력이 없고, 스토리는 짜임새가 없고. 그냥 음악만 좋은 영화.",0
...,...,...,...,...
2395,0,10408,한나라의 경제를 장기적으로 파탄으로 몰고간 역사를 배울수있음,0
2396,0,10542,"會者定離,,,",1
2397,0,12293,짜가리 영화가 별점 존나높네 짝퉁 클라스 보소6점이나 되네,0
2398,0,10240,어쨋거나 지루하잖아?,0
